In [38]:
# основные библиотеки для работы с данными
import pandas as pd
import numpy as np
import re

# инструменты визуализации
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
pd.set_option('display.max_columns', 30)

In [40]:
# загружаем датасет, который был частично обработан на этапе первого релиза
# логику обработки и принятия решений можно посмотреть в:
# realises/Release_1/EDA_part_1.ipynb
# Здесь работа с датасетом продолжается на том месте, на котором остановились
# на предыдущем этапе
# Датасеты "слили" в общий опять, и работали уже над одни объемом
df = pd.read_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/full_cleaned_data.csv')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23199 entries, 0 to 23198
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID  объявления       23199 non-null  int64  
 1   Количество комнат    22170 non-null  float64
 2   Метро                21893 non-null  object 
 3   Парковка             9871 non-null   object 
 4   Ремонт               20454 non-null  object 
 5   Балкон               15287 non-null  float64
 6   Окна                 16643 non-null  object 
 7   Высота потолков, м   11122 non-null  float64
 8   Мусоропровод         12743 non-null  object 
 9   Общая площадь, м2    23199 non-null  float64
 10  Тип дома             23199 non-null  object 
 11  Этажность дома       23199 non-null  int64  
 12  Этаж квартиры        23199 non-null  int64  
 13  Лоджия               15287 non-null  float64
 14  Санузел совмещенный  20559 non-null  float64
 15  Санузел раздельный   20559 non-null 

In [42]:
df.isnull().sum()

ID  объявления             0
Количество комнат       1029
Метро                   1306
Парковка               13328
Ремонт                  2745
Балкон                  7912
Окна                    6556
Высота потолков, м     12077
Мусоропровод           10456
Общая площадь, м2          0
Тип дома                   0
Этажность дома             0
Этаж квартиры              0
Лоджия                  7912
Санузел совмещенный     2640
Санузел раздельный      2640
Лифт пасс               5469
Лифт груз               5469
Можно с детьми          6287
Можно с животными      16102
Цена аренды                0
dtype: int64

In [43]:
# делали глубокую копию датафрейма, чтобы работать уже с ней
# планировалось поменять переменные и причесать все в красивый вид,
# но времени не хватило
df_tested = df.copy()

In [44]:
# если не указано в явном виде в объявлении, что можно с животными
# принимаем то, что с животными нельзя и заполняем колонку нулями

df_tested['Можно с животными'] = df_tested['Можно с животными'].fillna(0)

In [45]:
# по такой же логике коллегиально приняли решение заполнять пропуски, если нет
# явного указания в объявлении на то, что можно с детьми

df_tested['Можно с детьми'] = df_tested['Можно с детьми'].fillna(0)

In [46]:
# заполняем пропуски в колонках с лифтами
# принято решение, что если этажность дома больше 5, то добавляем по одному
# пассажирскому и одному грузовому лифту

df_tested.loc[
    (df_tested['Этажность дома'] > 5) & 
    (df_tested['Лифт пасс'].isna()) &
    (df_tested['Лифт груз'].isna()),
    ['Лифт пасс', 'Лифт груз']
    ] = 1

In [47]:
# оставшиеся пропуски заполняем нулями, т.к. если этажность дома меньше либо равна 5,
# принято решение, что лифтов в таком доме лифтов нет

df_tested[['Лифт пасс', 'Лифт груз']] = df_tested[['Лифт пасс', 'Лифт груз']].fillna(0)

In [48]:
# заполняем пропущенное значение санузла
# если не указан никакой из санузлов в объявлении,
# значит считаем, что один совмещенный

df_tested.loc[
    (df_tested['Санузел совмещенный'].isna()) &
    (df_tested['Санузел раздельный'].isna()),
    ['Санузел совмещенный']
    ] = 1

In [49]:
# заполняем оставшиеся пропуски нулевыми значениями
df_tested['Санузел совмещенный'] = df_tested['Санузел совмещенный'].fillna(0)
df_tested['Санузел раздельный'] = df_tested['Санузел раздельный'].fillna(0)

In [50]:
# если в изначальном объявлении не указано ни о лоджии, ни о балконе,
# считаем, что их нет и зануляем значение
# т.к. это конкурентное преимущество, и было бы явно указано о наличии балкона
# и лоджии, если бы они были

df_tested['Лоджия'] = df_tested['Лоджия'].fillna(0)
df_tested['Балкон'] = df_tested['Балкон'].fillna(0)

In [51]:
# если мусоропровод не указан, принимаем, что его нет и зануляем значение
df_tested['Мусоропровод'] = df_tested['Мусоропровод'].fillna(0)

In [52]:
# заполняем пропуски по высоте потолков
# используем медиану, игнорируя NaN значения при ее вычислении
# медиана принята как нечувствительная к выбросам величина

median_value = np.nanmedian(df_tested['Высота потолков, м'])
df_tested['Высота потолков, м'] = df_tested['Высота потолков, м'].fillna(median_value)

In [53]:
# заполняем пропущенные значения по наиболее часто встречающейся величине - моде
# моду смотрели в процессе работы через value_counts()

df_tested['Окна'] = df_tested['Окна'].fillna('Во двор')

In [54]:
# аналогично, по моде заполняем пропущенные значения ремонта

df_tested['Ремонт'] = df_tested['Ремонт'].fillna('Евроремонт')

In [55]:
# аналогично, по моде заполняем пропущенные значения для парковки
df_tested['Парковка'] = df_tested['Парковка'].fillna('наземная')

In [56]:
# заполнить пропуски метро по моде оказалось некорректно, т.к. небольшая разница
# между самым частым значением и последующими
# принято решение распределить по топ5 самым частым значениям

top_5_stations = df_tested['Метро'].value_counts().index[:5]

def fill_with_top_5(row, top_5):
    if pd.isna(row['Метро']):
        return np.random.choice(top_5)  # Случайный выбор станции из топ-5
    return row['Метро']


df_tested['Метро'] = df_tested.apply(lambda row: fill_with_top_5(row, top_5_stations), axis=1)


In [57]:
# количество комнат тоже нельзя однозначно описать по моде
# поэтому написали логику замены значений в зависимости от значений общей площади
# соотношение площади и количества комнат выявили собственным эмпирическим путем :)

df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] < 40), 'Количество комнат'] = 1
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 40) & (df_tested['Общая площадь, м2'] < 60), 'Количество комнат'] = 2
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 60) & (df_tested['Общая площадь, м2'] < 80), 'Количество комнат'] = 3
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 80) & (df_tested['Общая площадь, м2'] < 100), 'Количество комнат'] = 4
df_tested.loc[(df_tested['Количество комнат'].isna()) & (df_tested['Общая площадь, м2'] >= 100), 'Количество комнат'] = 5


In [58]:
# удалим полные дубликаты по всем колонкам, за исключением ID и цены аренды
# чтобы с одинаковыми наборами фичей у нас не получились разные цены и модель
# "не запуталась", как мы предположили

df_tested = df_tested.drop_duplicates(subset=[
    'Количество комнат',
    'Метро',
    'Парковка',
    'Ремонт',
    'Балкон',
    'Окна',
    'Высота потолков, м',
    'Мусоропровод',
    'Общая площадь, м2',
    'Тип дома',
    'Этажность дома',
    'Этаж квартиры',
    'Лоджия',
    'Санузел совмещенный',
    'Санузел раздельный',
    'Лифт пасс',
    'Лифт груз',
    'Можно с детьми',
    'Можно с животными'
    ]
                          )

In [59]:
# сохранение файла после препроцессинга для дальнейших манипуляций по преобразованию типов данных колонок
# df_tested.to_csv('/Users/alex_eyfler/ds_bootcamp/rent_real_estate_team_pandas/Alexander/data/almost_final_full_data.csv', index=False)